In [1]:
!pip install torch
!pip install tensorflow
!pip install --upgrade gym==0.19.0
!pip install gymnasium[atari]
!pip install keras
!pp install keras-r12
!pip install gym retro
!pip install gym[classic_control]
!pip install keras-rl
!pip install gym[accept-rom-license]

  Using cached gym-0.19.0.tar.gz (1.6 MB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.8/953.8 kB 10.9 MB/s eta 0:00:00
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl (2.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 44.3 MB/s eta 0:00:00
  Attempting uninstall: ale-py
    Found existing installation: ale-py 0.7.5
    Uninstalling ale-py-0.7.5:
      Successfully uninstalled ale-py-0.7.5
/bin/bash: line 1: pp: command not found


In [2]:
!pip install ale-py
!pip install retro

# DQNAgent TRAINING ON CARTPOLE


In [ ]:
import gymnasium as gym
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
import matplotlib.pyplot as plt
from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')


In [30]:

class DQN:
    def __init__(self, state_space, action_space):
        self.state_space = state_space
        self.action_space = action_space
        self.memory = []
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.gamma = 0.99
        self.learning_rate = 0.001
        self.model = self.build_model()

    def build_model(self):
        model = Sequential()
        model.add(Dense(24, input_shape=self.state_space, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_space, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.action_space)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])


In [29]:
# Create the environment
env = gym.make('CartPole-v1')

state_space = env.observation_space.shape
action_space = env.action_space.n

agent = DQN(state_space, action_space)

In [37]:
def train_env(env,agent,episodes=10,batch_size = 32):
  for episode in range(episodes):
      state = env.reset()
      done = False
      total_reward = 0

      while not done:
          # Preprocess state (if necessary)
          # (No need for np.expand_dims)

          # Choose action using epsilon-greedy policy
          if np.random.rand() < agent.epsilon:
              action = env.action_space.sample()  # Explore: choose a random action
          else:
              state_input = np.expand_dims(state, axis=0)  # Adding batch dimension
              # action = np.argmax(agent.model.predict(state_input))
              action = tf.convert_to_tensor(action) # Exploit: choose the best action

          # action= self.env.action_space.sample()  # selects an action from a sample space randomly
          # obs, reward, done, *info = self.env.step(action)


          # Take the action in the environment
          action= env.action_space.sample()
          next_state, reward, done, *info = env.step(action)

          # Preprocess next_state (if necessary)
          # (No need for np.expand_dims)

          # Store the experience in memory
          agent.remember(state, action, reward, next_state, done)

          # Update the state
          state = next_state
          total_reward += reward

          # Apply epsilon decay
          if agent.epsilon > agent.epsilon_min:
              agent.epsilon *= agent.epsilon_decay

          # Clear the previous plot and display the current frame
          # clear_output(wait=True)
          # plt.imshow(env.render())
          # plt.show()

          # Print episode statistics
      print(f"Episode: {episode+1}, Total Reward: {total_reward}, Epsilon: {agent.epsilon}")

# Close the environment after training
env.close()

In [38]:
# Create the environment
env = gym.make('CartPole-v1')

state_space = env.observation_space.shape
action_space = env.action_space.n

agent = DQN(state_space, action_space)
train_env(env,agent,episodes=50)

Episode: 1, Total Reward: 10.0, Epsilon: 0.9511101304657719
Episode: 2, Total Reward: 17.0, Epsilon: 0.8734200960253871
Episode: 3, Total Reward: 17.0, Epsilon: 0.8020760579717637
Episode: 4, Total Reward: 13.0, Epsilon: 0.7514768435208588
Episode: 5, Total Reward: 16.0, Epsilon: 0.6935613678313175
Episode: 6, Total Reward: 24.0, Epsilon: 0.6149486215357263
Episode: 7, Total Reward: 14.0, Epsilon: 0.5732736268885887
Episode: 8, Total Reward: 30.0, Epsilon: 0.4932355662165453
Episode: 9, Total Reward: 24.0, Epsilon: 0.43732904629000013
Episode: 10, Total Reward: 17.0, Epsilon: 0.4016064652978155
Episode: 11, Total Reward: 23.0, Epsilon: 0.3578751580867638
Episode: 12, Total Reward: 22.0, Epsilon: 0.32050833588933575
Episode: 13, Total Reward: 15.0, Epsilon: 0.29729358661854943
Episode: 14, Total Reward: 23.0, Epsilon: 0.2649210072611673
Episode: 15, Total Reward: 27.0, Epsilon: 0.231387331601191
Episode: 16, Total Reward: 23.0, Epsilon: 0.20619134658263935
Episode: 17, Total Reward: 10.

# DQNAgent TRAINING ON SPACE INVADERS

In [11]:
import gymnasium as gym
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
import matplotlib.pyplot as plt
from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')


In [12]:

class DQN:
    def __init__(self, state_space, action_space):
        self.state_space = state_space
        self.action_space = action_space
        self.memory = []
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.gamma = 0.99
        self.learning_rate = 0.001
        self.model = self.build_model()

    def build_model(self):
        model = Sequential()
        model.add(Dense(24, input_shape=self.state_space, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_space, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.action_space)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

# Initialize DQN agent
agent = DQN(state_space, action_space)




In [18]:

# Create the Space Invaders environment
env = gym.make('SpaceInvaders-v4',render_mode='rgb_array')

state_space = env.observation_space.shape
action_space = env.action_space.n

# Printing the results
print("State Space Shape:", state_space)
print("Action Space Size:", action_space)


State Space Shape: (210, 160, 3)
Action Space Size: 6


In [27]:
# Training loop
batch_size = 32
episodes = 10

for episode in range(episodes):
    state = env.reset()
    done = False
    total_reward = 0

    while not done:
        # Preprocess state (if necessary)
        # (No need for np.expand_dims)

        # Choose action using epsilon-greedy policy
        if np.random.rand() < agent.epsilon:
            action = env.action_space.sample()  # Explore: choose a random action
        else:
            state_input = np.expand_dims(state, axis=0)  # Adding batch dimension
            # action = np.argmax(agent.model.predict(state_input))
            action = tf.convert_to_tensor(action) # Exploit: choose the best action

        # action= self.env.action_space.sample()  # selects an action from a sample space randomly
        # obs, reward, done, *info = self.env.step(action)


        # Take the action in the environment
        action= env.action_space.sample()
        next_state, reward, done, *info = env.step(action)

        # Preprocess next_state (if necessary)
        # (No need for np.expand_dims)

        # Store the experience in memory
        agent.remember(state, action, reward, next_state, done)

        # Update the state
        state = next_state
        total_reward += reward

        # Apply epsilon decay
        if agent.epsilon > agent.epsilon_min:
            agent.epsilon *= agent.epsilon_decay

        # Clear the previous plot and display the current frame
        # clear_output(wait=True)
        # plt.imshow(env.render())
        # plt.show()

        # Print episode statistics
    print(f"Episode: {episode+1}, Total Reward: {total_reward}, Epsilon: {agent.epsilon}")

# Close the environment after training
env.close()

Episode: 1, Total Reward: 440.0, Epsilon: 0.00998645168764533
Episode: 2, Total Reward: 125.0, Epsilon: 0.00998645168764533
Episode: 3, Total Reward: 45.0, Epsilon: 0.00998645168764533
Episode: 4, Total Reward: 70.0, Epsilon: 0.00998645168764533
Episode: 5, Total Reward: 150.0, Epsilon: 0.00998645168764533
Episode: 6, Total Reward: 160.0, Epsilon: 0.00998645168764533
Episode: 7, Total Reward: 110.0, Epsilon: 0.00998645168764533
Episode: 8, Total Reward: 110.0, Epsilon: 0.00998645168764533
Episode: 9, Total Reward: 65.0, Epsilon: 0.00998645168764533
Episode: 10, Total Reward: 120.0, Epsilon: 0.00998645168764533


# DQNAgent TRAINING ON PACMAN

In [13]:
import numpy as np
import gym
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam  # Import the Adam optimizer

class DQN:
    def __init__(self, state_space, action_space):
        self.state_space = state_space
        self.action_space = action_space
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.gamma = 0.99
        self.learning_rate = 0.001
        self.model = self.build_model()

    def build_model(self):
        model = Sequential()
        model.add(Dense(24, input_shape=self.state_space, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_space, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))  # Use Adam optimizer
        return model

# Create the environment
env = gym.make('MsPacman-v0')

# Get state and action space dimensions
state_space = env.observation_space.shape
action_space = env.action_space.n

# Initialize DQN agent
agent = DQN(state_space, action_space)


In [14]:

# Create the MsPacman environment
env = gym.make('MsPacman-v0')

# Define the number of episodes
episodes = 10

for episode in range(episodes):
    state = env.reset()
    done = False
    total_reward = 0

    while not done:
        # Replace the following line with your specific algorithm to choose an action
        action = env.action_space.sample()

        # Take a step in the environment
        next_state, reward, done, _ = env.step(action)

        total_reward += reward
        state = next_state

    print(f"Episode {episode + 1}, Total Reward: {total_reward}")

# Don't forget to close the environment after you finish using it
env.close()


Episode 1, Total Reward: 190.0
Episode 2, Total Reward: 190.0
Episode 3, Total Reward: 380.0
Episode 4, Total Reward: 270.0
Episode 5, Total Reward: 100.0
Episode 6, Total Reward: 250.0
Episode 7, Total Reward: 260.0
Episode 8, Total Reward: 180.0
Episode 9, Total Reward: 120.0
Episode 10, Total Reward: 250.0


In [15]:
# Save the trained model
agent.model.save('pacman_dqn_model.h5')

In [17]:
import gym
import tensorflow as tf

# Create the MsPacman environment
env = gym.make('MsPacman-v0')

# Load the trained model
model = tf.keras.models.load_model('pacman_dqn_model.h5')

# Define the number of episodes
episodes = 20

for episode in range(episodes):
    state = env.reset()
    done = False
    total_reward = 0

    while not done:
        # Replace the following line with your specific algorithm to choose an action using the trained model
        # For example, in Deep Q-Network (DQN), you would use model.predict(state) to get the Q-values and select the action with the highest Q-value.
        action = env.action_space.sample()

        # Take a step in the environment
        next_state, reward, done, _ = env.step(action)

        total_reward += reward
        state = next_state

    print(f"Episode {episode + 1}, Total Reward: {total_reward}")

# Don't forget to close the environment after you finish using it
env.close()


Episode 1, Total Reward: 170.0
Episode 2, Total Reward: 340.0
Episode 3, Total Reward: 190.0
Episode 4, Total Reward: 130.0
Episode 5, Total Reward: 250.0
Episode 6, Total Reward: 330.0
Episode 7, Total Reward: 240.0
Episode 8, Total Reward: 200.0
Episode 9, Total Reward: 240.0
Episode 10, Total Reward: 230.0
Episode 11, Total Reward: 200.0
Episode 12, Total Reward: 260.0
Episode 13, Total Reward: 200.0
Episode 14, Total Reward: 240.0
Episode 15, Total Reward: 200.0
Episode 16, Total Reward: 300.0
Episode 17, Total Reward: 170.0
Episode 18, Total Reward: 140.0
Episode 19, Total Reward: 250.0
Episode 20, Total Reward: 230.0
